In [ ]:
%matplotlib inline
%config InlineBackend.figure_formats = {'svg',}

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from homogint import Integrator
from homogint.utils import time_step
import homogint.skeletons as sk

In [ ]:
%run plot_sphere.py

In [ ]:
rkmk4 = Integrator(sk.RKMK4())

In [ ]:
import tqdm

In [ ]:
def solve(vf,xs,stopping, maxit=10000):
    "Simple solver with stopping condition. The list xs is modified **in place**."
    for i in tqdm.tqdm(range(maxit)):
        if stopping(i,xs[-1]):
            break
        xs.append(rkmk4.step(vf, xs[-1]))

The nonautonomous equation is
\\[
x' = v(t) \times x
\\]
where 
\\[
v(t) = [0, 1, t]
\\]

This can be rewritten in an autonomous equation in \\(\mathbf{R}^5\\) as
\\[
X' = \begin{bmatrix}
[v(X_4)\times] & 0 & 0\\
0 & 0 & 1\\
0 & 0 & 0
\end{bmatrix}
X
\\]
Where \\([v\times]\\) denotes the skew symmetric matrix corresponding to the linear operator \\(x \mapsto v \times x\\).

In [ ]:
def get_field(time_dependent):
    """
    Create a vector field from a time dependent vector valued function.
    """
    def field(x):
        J = np.zeros([5,5])
        t = x[-2]
        v = time_dependent(t)
        J[0,1] = -v[2]
        J[0,2] = v[1]
        J[1,2] = -v[0]
        J -= J.T
        J[-2,-1] = 1.
        return J
    return field

In [ ]:
def timedep_field(t):
    return t*np.array([0.,0.,1]) + np.array([0,1,0])

In [ ]:
xs = [np.array([0.,0,1,-10,1])]
dt = .02
solve(time_step(dt)(get_field(timedep_field)),xs,lambda i,x:x[-2]>10)
axs = np.array(xs)

In [ ]:
def plot2(axs):
    plt.plot(axs[0,0],axs[0,1],'o')
    plt.plot(axs[:,0], axs[:,1],marker='.')

In [ ]:
plt.axis('equal')
plot2(axs)

In [ ]:
fig = plt.figure(figsize=(15,10))
ax = plot_sphere()
s = slice(None)
ax.plot(axs[s,0],axs[s,1],axs[s,2],lw=2,marker='.',color=['black','blue'][0], alpha=[1.,0.2][0])
ax.view_init(30,-100)
